In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agriculture-data/cleaned_text_data.csv


In [2]:
# GPU-accelerated HDBSCAN + UMAP
!pip install cudf-cu12 dask-cudf-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cuml-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cugraph-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install cupy-cuda12x -f https://pip.cupy.dev/aarch64

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 176.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 145.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 GB 1.5 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 168.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 855.3/855.3 kB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 MB 69.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 170.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
data = pd.read_csv('/kaggle/input/agriculture-data/cleaned_text_data.csv')

In [4]:
data.head(10)

,text,date,file_name
0,With a big drop in Annual Investment Allowance...,2020-10-27,20.json
1,Though the moisture was welcomed in much of th...,2020-10-27,61.json
2,Hello and welcome to the ZimmCast. This podcas...,2020-10-27,36.json
3,Ecological monitoring is the recording of biol...,2020-10-27,41.json
4,Farmers have been encouraged to develop a cont...,2020-10-27,16.json
5,Farming 18 blocks of land spread over the loca...,2020-10-27,57.json
6,Published: Just now DecResearch Inc. is one of...,2020-10-27,6.json
7,Global Modified Starch for Animal Feed Market ...,2020-10-27,7.json
8,"Sometimes, I sit down to write these columns w...",2020-10-27,56.json
9,The Suckler Beef Climate Group has published i...,2020-10-27,17.json


In [4]:
# import nltk
# nltk.download('punkt')

# from nltk.tokenize import word_tokenize
# data['token_count'] = data['text'].apply(lambda x: len(word_tokenize(x)))

In [5]:
data.head(5)

,text,date,file_name
0,With a big drop in Annual Investment Allowance...,2020-10-27,20.json
1,Though the moisture was welcomed in much of th...,2020-10-27,61.json
2,Hello and welcome to the ZimmCast. This podcas...,2020-10-27,36.json
3,Ecological monitoring is the recording of biol...,2020-10-27,41.json
4,Farmers have been encouraged to develop a cont...,2020-10-27,16.json


In [4]:
data.shape

(22814, 3)

In [5]:
from typing import List
import re

In [6]:
def chunk_document(text: str, chunk_size: int = 100) -> List[str]:
    try:
        # Basic sentence splitting using common sentence endings
        sentences = re.split(r'(?<=[.!?])\s+', text)
        
        chunks = []
        current_chunk = []
        current_size = 0
        
        for sentence in sentences:
            words = sentence.split()
            if current_size + len(words) <= chunk_size:
                current_chunk.append(sentence)
                current_size += len(words)
            else:
                if current_chunk:
                    chunks.append(' '.join(current_chunk))
                current_chunk = [sentence]
                current_size = len(words)
        
        if current_chunk:
            chunks.append(' '.join(current_chunk))
        
        return chunks
    except Exception as e:
        print(f"Error chunking document: {e}")
        return [text]  # Return original text if chunking fails

In [7]:
data = data[['text']]

In [8]:
data.shape

(22814, 1)

In [9]:
chunked_data = []

for idx, row in data.iterrows():
    text = row['text']
    chunks = chunk_document(text, chunk_size=100)  # Adjust chunk_size as needed
    
    for chunk in chunks:
        chunked_data.append({'original_index': idx, 'chunk': chunk})

In [10]:
chunked_df = pd.DataFrame(chunked_data)

In [11]:
print("Shape of chunked DataFrame:", chunked_df.shape)

Shape of chunked DataFrame: (165322, 2)


In [12]:
!pip install llama-cpp-python --q
!pip install bertopic --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 15.3 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 61.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3

In [13]:
!wget https://huggingface.co/TheBloke/OpenHermes-2.5-Mistral-7B-GGUF/resolve/main/openhermes-2.5-mistral-7b.Q4_K_M.gguf
# !wget https://huggingface.co/TheBloke/dolphin-2.7-mixtral-8x7b-GGUF/resolve/main/dolphin-2.7-mixtral-8x7b.Q3_K_M.gguf

--2025-04-23 04:24:16--  https://huggingface.co/TheBloke/OpenHermes-2.5-Mistral-7B-GGUF/resolve/main/openhermes-2.5-mistral-7b.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.238.136.32, 18.238.136.129, 18.238.136.112, ...
Connecting to huggingface.co (huggingface.co)|18.238.136.32|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/65/3b/653b1cc7a4558cd22c0ac02df471105878a080cb5694622ccd566697c574592f/cd58120326971c71c0590f6b7084a0744e287ce9c67275d8b4bf34a5947d950b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27openhermes-2.5-mistral-7b.Q4_K_M.gguf%3B+filename%3D%22openhermes-2.5-mistral-7b.Q4_K_M.gguf%22%3B&Expires=1745385856&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NTM4NTg1Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzY1LzNiLzY1M2IxY2M3YTQ1NThjZDIyYzBhYzAyZGY0NzExMDU4NzhhMDgwY2I1Njk0NjIyY2NkNTY2Njk3YzU3NDU5MmYvY2Q1ODEyMDMyNjk3MW

In [14]:
from llama_cpp import Llama
llm = Llama(model_path="openhermes-2.5-mistral-7b.Q4_K_M.gguf", n_gpu_layers=-1, n_ctx=4096, stop=["Q:", "\n"])

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from openhermes-2.5-mistral-7b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = teknium_openhermes-2.5-mistral-7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 l

In [15]:
from bertopic.representation import LlamaCPP

prompt = """ Q:
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the above information, can you give a short label of the topic of at most 5 words?
A:
"""

representation_model = {
    "LLM": LlamaCPP(llm, prompt=prompt)
}

2025-04-23 04:25:15.664538: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745382315.853862      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745382315.907398      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [16]:
docs= chunked_df['chunk']

In [17]:
len(docs)

165322

In [18]:
from sentence_transformers import SentenceTransformer
from cuml.manifold import UMAP
from cuml.cluster import HDBSCAN
# from umap import UMAP
# from hdbscan import HDBSCAN

# Pre-calculate embeddings
embedding_model = SentenceTransformer("BAAI/bge-small-en")
embeddings = embedding_model.encode(docs, show_progress_bar=True)

# Pre-reduce embeddings for visualization purposes
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/5167 [00:00<?, ?it/s]

[2025-04-23 04:33:02.778] [CUML] [info] build_algo set to brute_force_knn because random_state is given


In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
# Define sub-models
# umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
umap_model = UMAP(n_neighbors=10, n_components=3, min_dist=0.1, metric='cosine', random_state=42, low_memory=True)
# hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
hdbscan_model = HDBSCAN(min_cluster_size=100, min_samples=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(2, 2))

[2025-04-23 04:33:12.860] [CUML] [info] Unused keyword parameter: low_memory during cuML estimator initialization
[2025-04-23 04:33:12.861] [CUML] [info] build_algo set to brute_force_knn because random_state is given


In [21]:
from bertopic import BERTopic

topic_model = BERTopic(

  # Sub-models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,vectorizer_model = vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(docs, embeddings)

2025-04-23 04:39:42,782 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-23 04:40:01,149 - BERTopic - Dimensionality - Completed ✓
2025-04-23 04:40:01,160 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-23 04:40:09,700 - BERTopic - Cluster - Completed ✓
2025-04-23 04:40:09,736 - BERTopic - Representation - Fine-tuning topics using representation models.
  0%|          | 0/183 [00:00<?, ?it/s]llama_perf_context_print:        load time =  146424.35 ms
llama_perf_context_print: prompt eval time =  146423.54 ms /   808 tokens (  181.22 ms per token,     5.52 tokens per second)
llama_perf_context_print:        eval time =    1975.28 ms /     6 runs   (  329.21 ms per token,     3.04 tokens per second)
llama_perf_context_print:       total time =  148402.73 ms /   814 tokens
  1%|          | 1/183 [02:28<7:30:10, 148.41s/it]Llama.generate: 43 prefix-match hit, remaining 935 prompt tokens to eval
llama_perf_context_print:        load 

In [25]:
topic_info = topic_model.get_topic_info()

In [26]:
topic_info.head()

,Topic,Count,Name,Representation,LLM,Representative_Docs
0,-1,86524,-1_climate change_plant based_supply chain_far...,"[climate change, plant based, supply chain, fa...","[Sustainable Agriculture, , , , , , , , , ]",[Although he has no data to show yields have i...
1,0,4451,0_soy sauce_olive oil_sesame oil_gluten free,"[soy sauce, olive oil, sesame oil, gluten free...","[[INST]\n\nShort Label: Asian Fusion Recipes, ...",[Print Miso Satay Eggs Total Time 15 minutes S...
2,1,3788,1_john deere_new holland_self propelled_massey...,"[john deere, new holland, self propelled, mass...","[Tractor Specs, , , , , , , , , ]",[Vital stats Valtra G135 HiTech Max power 135h...
3,2,3349,2_land management_environmental land_sustainab...,"[land management, environmental land, sustaina...","[Environmental Land Management Schemes, , , , ...",[A 'lower-than-expected' level of interest in ...
4,3,3079,3_dry matter_milk production_dm ha_body condition,"[dry matter, milk production, dm ha, body cond...","[Grass Silage and Dairy Farming, , , , , , , ,...","[""We need to calve the herd quickly and effici..."


In [27]:
# Save the topic information to a CSV file
topic_info.to_csv('topic_info.csv', index=False)

In [28]:
topic_model.get_topic(1, full=True)

{'Main': [('john deere', 0.006787830532466935),
  ('new holland', 0.005123795783297954),
  ('self propelled', 0.0035550248731217086),
  ('massey ferguson', 0.003355089055552476),
  ('litre cylinder', 0.0033040858991294692),
  ('wheel drive', 0.003288685160837065),
  ('yes yes', 0.003129890573648985),
  ('litres min', 0.002568013401614885),
  ('disc coulters', 0.0025250179244172433),
  ('second hand', 0.0025107869820325335)],
 'LLM': [('Tractor Specs', 1),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0)]}

In [29]:
fig1 = topic_model.visualize_topics()
fig1.write_html("intertopic_distance_map2.html")

In [30]:
fig4 = topic_model.visualize_heatmap(n_clusters=20, top_n_topics=100)
fig4.write_html("similarity_matrix.html")

In [31]:
fig3 = topic_model.visualize_hierarchy(top_n_topics=50, width=700)
fig3.write_html("hierarchical_clustering.html")